In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
from numpy import concatenate
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import math
from tqdm.notebook import tqdm
tf.random.set_seed(0)
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape

In [ ]:
dataframe = pd.read_excel('D:\\Desktop\\akshat\\Internship_Stuff\\Dataset\\Forecasting_Data_Values\\test_5412_2.xlsx')
print(dataframe)

In [ ]:
# Change the columns based on factors taken for Forecasting
dataframe=dataframe.drop(columns=['NINO'])

# Normalizing Data 

In [ ]:
values=dataframe.values
print(values)

In [ ]:
values_spi=values[:,1]
print(values_spi.shape,type(values_spi))

In [ ]:
values_spi=values_spi.reshape(-1,1)
print(values_spi.shape)

In [ ]:
scaler=MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values_spi)

In [ ]:
v=values
v[:,1]=scaled[:,0]
print(v)
values1=v

In [ ]:
df_scaled = pd.DataFrame(values1)
print(df_scaled)

# Adding Lookback

In [ ]:
dX = []
dY = []
n_future = 1
n_past = 6 #No of Past values for forecasting
for i in range(n_past, len(df_scaled) - n_future +1):
    dX.append(values1[i - n_past:i, 1:dataframe.shape[1]])
    dY.append(values1[i + n_future - 1:i + n_future,1])
dX, dY = np.asarray(dX), np.asarray(dY)

In [ ]:
print(len(dX),dX.shape,len(dY),dY.shape)

In [ ]:
dX = np.asarray(dX).astype(np.float32)
dY = np.asarray(dY).astype(np.float32)

# Training and Testing Split

In [ ]:
train_size = int(len(dX)-24)
test_size = len(dX) - train_size
trainX, testX = dX[0:train_size,:], dX[train_size:len(dX),:]
print(len(trainX),trainX.shape,len(testX),testX.shape)

In [ ]:
train_size = int(len(dY)-24)
test_size = len(dY) - train_size
trainY, testY = dY[0:train_size,:], dY[train_size:len(dY),:]
print(len(trainY),trainY.shape,len(testY),testY.shape)

# Callback to prevent overfitting(Optional)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('loss')<0.0006):
            print("\nLoss is low so cancelling training!")
            self.model.stop_training = True
callbacks = myCallback()

# LSTM Model

In [ ]:
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=False))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
history = model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=1)

In [ ]:
trainPredict = model.predict(trainX, batch_size=1)
trainPredict = scaler.inverse_transform(trainPredict)
trainY_2 = scaler.inverse_transform(trainY)
model.reset_states()
testPredict = model.predict(testX,batch_size=1)
testPredict = scaler.inverse_transform(testPredict)
testY_2 = scaler.inverse_transform(testY)

In [ ]:
plt.plot(trainY_2)
plt.plot(trainPredict)
plt.legend(['observed','predicted'])
plt.show()

In [ ]:
#extra stuff to  make proper graphs
z=[]
for i in range(0,25):
    z.append(i)
print(z)

In [ ]:
plt.plot(testY_2)
plt.plot(testPredict)
plt.xticks(z)
plt.legend(['observed','predicted'])
plt.ylabel('SPI')
plt.xlabel('Month')
plt.show()

# Cheking the error scores

In [ ]:
#RMSE
trainScore = math.sqrt(mean_squared_error(trainY_2[:,0], trainPredict[:,0]))
print('Train Score: %.2f' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY_2[:,0], testPredict[:,0]))
print('Test Score: %.2f' % (testScore))

In [ ]:
#Preliminary stuff for other errors
v=testY_2.copy()
ob=[]
for i in v:
    ob.append(i[0])
print(ob)
v=testPredict.copy()
pred=[]
for i in v:
    pred.append(i[0])
print(pred)

In [ ]:
#Correlation score
r = np.corrcoef(ob,pred)
print(" correlation:")
print(r)

In [ ]:
# Mean Absolute Error
m = mae(ob,pred)
print('MAE:'+str(m))

In [ ]:
#Mean Absolute Percentage Error
print('MAPE: '+str(mape(ob,pred)))

In [ ]:
#Percentage fo Droughts predictied and Percentage of Predictions in 1 Drought Catecory
dob=0
dp=0
diffok=0
for i in range(0,len(ob)):
    if ob[i]<=0:
        dob+=1
        if pred[i]<=0:
            dp+=1
            if abs(ob[i]-pred[i])<0.5:
                diffok+=1
print("Percentage of droughts predicted: "+ str((dp/dob)*100))
print("Percentage of Predicted drought with low disparity: "+ str((diffok/dp)*100))

# Extra Stuff for Feature Importance

COMPUTE_LSTM_IMPORTANCE = True

COLS = list(df_scaled.columns)
print(COLS)

trainX[:,0]

print(testY.shape)

testX[:,:,0]

values1=values1[:,1:]
print(values1)

if COMPUTE_LSTM_IMPORTANCE:
            results = []
            print(' Computing LSTM feature importance...')
            
            # COMPUTE BASELINE (NO SHUFFLE)
            oof_preds = model.predict(testX, verbose=0).squeeze() 
            print(oof_preds.shape)
            print(testY.shape)
            baseline_mae = np.mean(np.abs( oof_preds-testY ))
            results.append({'feature':'BASELINE','mae':baseline_mae})           

            for k in tqdm(range(len(COLS)-1)):
                
                # SHUFFLE FEATURE K
                save_col = values1[train_size:len(dX),k].copy()
                print(save_col.shape)
                np.random.shuffle(testX[:,k])
                        
                # COMPUTE OOF MAE WITH FEATURE K SHUFFLED
                oof_preds = model.predict(testX, verbose=0).squeeze()
                print(oof_preds.shape)
                print(testY.shape)
                mae = np.mean(np.abs( oof_preds-testY ))
                results.append({'feature':COLS[k],'mae':mae})
                testY = save_col
         
            # DISPLAY LSTM FEATURE IMPORTANCE
            print()
            df = pd.DataFrame(results)
            df = df.sort_values('mae')
            plt.figure(figsize=(10,20))
            plt.barh(np.arange(len(COLS)),df.mae)
            plt.yticks(np.arange(len(COLS)),df.feature.values)
            plt.title('LSTM Feature Importance',size=16)
            plt.ylim((-1,len(COLS)+1))
            plt.plot([baseline_mae,baseline_mae],[-1,len(COLS)], '--', color='orange',
                     label=f'Baseline OOF\nMAE={baseline_mae:.3f}')
            plt.ylabel('Feature',size=14)
            plt.legend()
            plt.show()